In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time 
import json

In [2]:
roi = pd.read_csv('roiimdb.csv')

In [3]:
roi['imdbcode']=roi.imdblink.apply(lambda x : x[7:].rstrip('/'))

In [4]:
roi.head()

,Unnamed: 0,level_0,index,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,dom_roi,world_roi,release_month,release_day,release_year,season,imdblink,imdbcode
0,0,0,1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,-0.412899,1.546673,May,20,2011,spring,/title/tt1298650/,tt1298650
1,1,1,2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.877822,-0.572108,Jun,7,2019,summer,/title/tt6565702/,tt6565702
2,2,2,3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,0.388403,3.243841,May,1,2015,spring,/title/tt2395427/,tt2395427
3,3,3,4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,0.956408,3.153696,Dec,15,2017,winter,/title/tt2527336/,tt2527336
4,4,4,5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2.060988,5.710167,Dec,18,2015,winter,/title/tt2488496/,tt2488496


In [145]:
df=pd.DataFrame.from_dict(imdbdata, orient='index')
df.head()

,0,1,2,3,4,5,6,7,8,9
tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,2h 16min,"[Action, Adventure, Fantasy]",20 May 2011,USA,6.6,None,NaN
tt6565702,X-Men: Dark Phoenix,2019,PG-13,1h 53min,"[Action, Adventure, Sci-Fi]",7 June 2019,USA,5.8,None,NaN
tt2395427,Avengers: Age of Ultron,2015,PG-13,2h 21min,"[Action, Adventure, Sci-Fi]",1 May 2015,USA,7.3,None,NaN
tt2527336,Star Wars: Episode VIII - The Last Jedi,2017,PG-13,2h 32min,"[Action, Adventure, Fantasy]",15 December 2017,USA,7,None,NaN
tt2488496,Star Wars: Episode VII - The Force Awakens,2015,PG-13,2h 18min,"[Action, Adventure, Sci-Fi]",18 December 2015,USA,7.9,None,NaN


In [292]:
sum([True for movie in badmovies if movie in df.index])

54

In [143]:
#Attempt1 on IMDB primary data
imdbdata = {}
badmovies = []
dirtymovies= []
for key in roi.imdbcode:
    time.sleep(random.randrange(0,2))
    moviedata = []
    url='https://www.imdb.com/title/'+key+'/'
    page = requests.get(url)
    if page.status_code != requests.codes.ok:
        badmovies.append(key)
        pass
    soup = BeautifulSoup(page.content, 'html.parser')
    #Title, Year, Rating, Runtime, Genre(s), Release Date, IMDB_SCORE
    try:
        div1 = soup.find('div', class_='title_wrapper')
        #Title,Year
        titleyearstring= div1.find('h1').text
        title=titleyearstring[:-7].strip()
        moviedata.append(title)
        year=titleyearstring[-6:-2].strip()
        moviedata.append(year)
        #Rating, Runtime, Genre(s), Release Date (split country)
        subinfo=div1.find('div', class_='subtext').text.split('|')
        cleansub= [x.strip().strip('\n') for x in subinfo]
        try:
            moviedata.append(cleansub[0])
            moviedata.append(cleansub[1])
            moviedata.append([x.lstrip(' \n') for x in cleansub[2].split(',')])
            date, country = cleansub[3].split('(')
            moviedata.append(date.strip())
            moviedata.append(country[:-1])
        except (ValueError, IndexError):
            if len(cleansub)!=4:
                dirtymovies.append(key)
            for item in cleansub:
                moviedata.append(item)
        #IMDB SCORE
        score = float(soup.find('span', itemprop='ratingValue').text)
        moviedata.append(score)
    except AttributeError:
        print('Unsuccessful: '+key)
        badmovies.append(key)
        pass
    imdbdata[key]=moviedata
    
if len(imdbdata)==len(roi.imdbcode):
    print('movies scraped successfully!')

else:
    print('completed but ' + str(len(bad_movies)) + ' were not imported!')    


Unsuccessful: tt8805968
Unsuccessful: m0001073
Unsuccessful: m0000187
Unsuccessful: tt7813262
Unsuccessful: m3119473
Unsuccessful: tt9382818
Unsuccessful: tt3062026
Unsuccessful: m0000203
Unsuccessful: tt3006462
Unsuccessful: tt1397132
Unsuccessful: m0004266
Unsuccessful: m0502342
Unsuccessful: tt8080292
Unsuccessful: m0290556
Unsuccessful: tt3070498
Unsuccessful: tt5421212
Unsuccessful: m0000568
Unsuccessful: m0396079
Unsuccessful: m0908094
Unsuccessful: m0005522
Unsuccessful: m0000827
Unsuccessful: tt4943216
Unsuccessful: tt5275280
Unsuccessful: m2442289
Unsuccessful: m3507289
Unsuccessful: tt11641040
Unsuccessful: m3117320
Unsuccessful: m0001576
Unsuccessful: tt2355062
Unsuccessful: m0002121
Unsuccessful: m0330687
Unsuccessful: m0000206
Unsuccessful: m0738042
Unsuccessful: tt10177850


NameError: name 'bad_movies' is not defined

In [153]:
imdbbaddf=pd.DataFrame(badmovies)
imdbbaddf.to_csv('listofbadmoviesimdb.csv')

In [154]:
imdbroughdf=pd.DataFrame(dirtymovies)
imdbroughdf.to_csv('listofdirtymoviesimdb.csv')

In [166]:
df[4].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1652, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Comedy, Drama, Romance]          79
[Action, Adventure, Sci-Fi]       68
[Animation, Adventure, Comedy]    68
[Comedy, Drama]                   67
[Drama]                           66
                                  ..
[27 August 2015 (USA)]             1
[Adventure, Mystery, Sci-Fi]       1
[20 September 2015 (Germany)]      1
[Action, Horror, Romance]          1
[Adventure, Comedy, Western]       1
Name: 4, Length: 271, dtype: int64

In [ ]:
#webscraper test
imdb_links=[]
bad_movies=[]

for movie in test2.movie:
    time.sleep(random.randrange(0,3))
    title=movie
    if ':' in title:
        title=title.replace(':','%3A')
    if ',' in title:
        title=title.replace(',','%2C')
    query = title.replace(' ','+')
    url='https://www.imdb.com/find?q='+query+'&ref_=nv_sr_sm'
    page = requests.get(url)
    if page.status_code == requests.codes.ok:
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            imdb_url = soup.find('td', class_='result_text').find('a').attrs['href']
            imdb_links.append(imdb_url)
        except AttributeError:
            bad_movies.append(movie)
            pass
    else:
        bad_movies.append(movie)
        
if len(imdb_links)==len(test2):
    print('movies scraped successfully!')
else:
    print('completeed but ' + str(len(bad_movies)) + ' were not imported!')

In [138]:
#Title, Rating, Runtime, Genre(s), Release Date, IMDB_SCORE
badmovies=[]
imdbdata= {}
moviedata=[] 
key = roi.imdbcode[random.randrange(0,len(roi.imdbcode))]
url = 'https://www.imdb.com/title/'+key+'/'
page = requests.get(url)
print(url)
if page.status_code != requests.codes.ok:
    print('Unsuccessful connection')
        #USE GOOGLE WEB CACHE HERE?
        #If unsuccessful:
            #badmovielinks.append(link)
            #imdbdata.append(moviedata)
            #continue
soup = BeautifulSoup(page.content, 'html.parser')
#Title, Year, Rating, Runtime, Genre(s), Release Date(split country), IMDB_SCORE
try:
    div1 = soup.find('div', class_='title_wrapper')
    #Title,Year
    titleyearstring= div1.find('h1').text
    title=titleyearstring[:-7].strip()
    moviedata.append(title)
    year=titleyearstring[-6:-2].strip()
    moviedata.append(year)
    #Rating, Runtime, Genre(s), Release Date (split country)
    subinfo=div1.find('div', class_='subtext').text.split('|')
    cleansub= [x.strip().strip('\n') for x in subinfo]
    moviedata.append(cleansub[0])
    moviedata.append(cleansub[1])
    moviedata.append([x.lstrip(' \n') for x in cleansub[2].split(',')])
    date, country = cleansub[3].split('(')
    moviedata.append(date.strip())
    moviedata.append(country[:-1])
    #IMDB SCORE
    score = float(soup.find('span', itemprop='ratingValue').text)
    moviedata.append(score)

except AttributeError:
    print('Unsuccessful: '+key)
    badmovies.append(key)
imdbdata[key]= moviedata
print(imdbdata)

https://www.imdb.com/title/tt4030600/
{'tt4030600': ['The Bye Bye Man', '2017', 'PG-13', '1h 36min', ['Drama', 'Fantasy', 'Horror'], '13 January 2017', 'USA', 4.2]}


In [176]:
#ATTEMPT ON SCORE/SECONDARY INFO
scorebadmovies=[]
scoredata= {}
scoredirtymovies= []
for key in roi.imdbcode:
    if key in badmovies:
        print(key+' avoided!')
        continue
    time.sleep(random.randrange(0,2))
    moviedata = []
    url='https://www.imdb.com/title/'+key+'/'
    page = requests.get(url)
    if page.status_code != requests.codes.ok:
        print('Unsuccessful: '+key)
        scorebadmovies.append(key)
        pass
    soup = BeautifulSoup(page.content, 'html.parser')
    #Title, Year, Rating, Runtime, Genre(s), Release Date, IMDB_SCORE
    try:
        messy=soup.find('div', class_='plot_summary_wrapper').find_all('div', class_='credit_summary_item')
        #directors
        try:
            director = messy[0].text.split(':')[1].split(',')
            cleandirectors = [x.strip().lstrip('\n') for x in director]
        except IndexError:
            scoredirtymovies.append(key)
            print('bad directors: '+key)
            cleandirectors=messy.text
        directorid = messy[0].find('a').attrs['href'][6:-1]
        #writers
        try:
            writers = messy[1].text.split('|')[0].split(':')[1].split(',')
            cleanwriters=[x.strip().lstrip('\n') for x in writers]
        except IndexError:
            scoredirtymovies.append(key)
            print('bad writers: '+key)
            cleanwriters=messy.text
        writersid= [a.attrs['href'][6:-1] for a in messy[1].find_all('a')]
        #stars
        try:
            stars= messy[2].text.split('|')[0].split(':')[1].split(',')
            cleanstars=[x.strip().lstrip('\n') for x in stars]
        except IndexError:
            scoredirtymovies.append(key)
            print('bad actors: '+key)
            cleanstars=messy.text
        starsid=[a.attrs['href'][6:-1] for a in messy[-1].find_all('a')][:-1]
        #adding info to moviedata
        moviedata.append(cleandirectors)
        moviedata.append(directorid)
        moviedata.append(cleanwriters)
        moviedata.append(writersid)
        moviedata.append(cleanstars)
        moviedata.append(starsid)
        #metascore
        metascore = int(soup.find('div', class_='metacriticScore').text)
        moviedata.append(metascore)
        creditdata[key]=moviedata
    except AttributeError:
        print('Unsuccessful: '+key)
        scorebadmovies.append(key)
        continue
    #assigning info
    scoredata[key]=moviedata
    
if len(scoredata)==len(roi.imdbcode):
    print('movies scraped successfully!')

else:
    print('completed but ' + str(len(scorebadmovies)) + ' were unsuccesful!')    

tt8805968 avoided!
m0001073 avoided!
m0000187 avoided!
bad actors: tt2676292
Unsuccessful: tt2676292
tt7813262 avoided!
m3119473 avoided!
tt9382818 avoided!
Unsuccessful: tt1680310
Unsuccessful: tt1571403
tt3062026 avoided!
bad writers: tt7062794
Unsuccessful: tt7062794
m0000203 avoided!
tt3006462 avoided!
tt1397132 avoided!
m0004266 avoided!
Unsuccessful: tt0032553
m0502342 avoided!
tt8080292 avoided!
m0290556 avoided!
tt3070498 avoided!
tt5421212 avoided!
Unsuccessful: tt0940656
m0000568 avoided!
m0396079 avoided!
m0908094 avoided!
Unsuccessful: tt2631186
bad actors: tt6156584
Unsuccessful: tt6156584
Unsuccessful: tt2133584
Unsuccessful: tt1817771
bad actors: tt6932244
Unsuccessful: tt6932244
bad actors: tt0387199
Unsuccessful: tt0387199
Unsuccessful: tt4849438
Unsuccessful: tt1711018
bad actors: tt5114356
Unsuccessful: tt5114356
bad actors: tt0075488
Unsuccessful: tt0075488
Unsuccessful: tt11490170
m0005522 avoided!
bad writers: tt7535844
Unsuccessful: tt7535844
m0000827 avoided!
Un

NameError: name 'scoresdata' is not defined

In [ ]:
backup= scoredata

In [177]:
scoredf= pd.DataFrame.from_dict(scoredata, orient='index')

In [178]:
scoredf.to_csv('roughscore.csv')

In [179]:
scorebaddf=pd.DataFrame(scorebadmovies)

In [258]:
scoredirtydf=pd.DataFrame(scoredirtymovies)

In [181]:
scorebaddf.to_csv('scorebad.csv')

In [182]:
scoredirtydf.to_csv('scoredirty.csv')

In [183]:
scoredf.shape

(1584, 7)

In [260]:
scoredirtydf['title'] = [" ".join(roi[roi.imdbcode==key].movie.to_string().split()[1:]) for key in scoredirtydf[0]]

In [262]:
scoredirtydf.to_csv('scoredirty.csv')

In [265]:
scorebaddf['title'] = [" ".join(roi[roi.imdbcode==key].movie.to_string().split()[1:]) for key in scorebaddf[0]]

In [268]:
scorebaddf.to_csv('scorebad.csv')

In [272]:
imdbbaddf['title'] = [" ".join(roi[roi.imdbcode==key].movie.to_string().split()[1:]) for key in imdbbaddf[0]]

In [275]:
imdbbaddf.to_csv('imdbbadmovies.csv')

In [277]:
imdbroughdf['title'] = [" ".join(roi[roi.imdbcode==key].movie.to_string().split()[1:]) for key in imdbroughdf[0]]

In [278]:
imdbroughdf.head()

,0,title
0,tt8805968,Guardians of the Galaxy Vol 2
1,tt2676292,Cowboys and Aliens
2,tt7813262,Fast and Furious 6
3,tt9382818,"The Twilight Saga: Breaking Dawn, Part 2"
4,tt1680310,Harry Potter and the Deathly Hallows: Part II


In [135]:
#secondary info draft
badmovies=[]
creditdata= {}
moviedata=[] 
key = roi.imdbcode[random.randrange(0,len(roi.imdbcode))]
url = 'https://www.imdb.com/title/'+key+'/'
page = requests.get(url)
if page.status_code != requests.codes.ok:
    print('Unsuccessful connection')
soup=BeautifulSoup(page.content, 'html.parser')
messy=soup.find('div', class_='plot_summary_wrapper').find_all('div', class_='credit_summary_item')
#directors
director = messy[0].text.split(':')[1].split(',')
cleandirectors = [x.strip().lstrip('\n') for x in director]
directorid = messy[0].find('a').attrs['href'][6:-1]
#writers
writers = messy[1].text.split('|')[0].split(':')[1].split(',')
writersid= [a.attrs['href'][6:-1] for a in messy[1].find_all('a')]
cleanwriters=[x.strip().lstrip('\n') for x in writers]
#stars
stars= messy[2].text.split('|')[0].split(':')[1].split(',')
starsid=[a.attrs['href'][6:-1] for a in messy[2].find_all('a')][:-1]
cleanstars=[x.strip().lstrip('\n') for x in stars]
moviedata.append(cleandirectors)
moviedata.append(directorid)
moviedata.append(cleanwriters)
moviedata.append(writersid)
moviedata.append(cleanstars)
moviedata.append(starsid)
#metascore
metascore = int(soup.find('div', class_='metacriticScore').text)
moviedata.append(metascore)
print('Metascore= '+str(metascore))
creditdata[key]=moviedata


https://www.imdb.com/title/tt5140878/
Metascore= 62


In [184]:
#ATTEMPT FOR CAST SCRAPE
castbadmovies=[]
castdata = {}
castdirtymovies=[]
for key in roi.imdbcode:
    if key in [badmovies, scorebadmovies]:
        print(key+' avoided!')
        castbadmovies.append(key)
        continue
    time.sleep(random.randrange(0,2))
    moviedata = []
    url='https://www.imdb.com/title/'+key+'/'
    page = requests.get(url)
    if page.status_code != requests.codes.ok:
        print('Unsuccessful: '+key)
        castbadmovies.append(key)
        pass
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        messy = soup.find('table',class_='cast_list'a).find_all('td', class_=False)
        cast= [actor.text.replace('\n','').strip() for actor in messy]
        moviedata.append(cast)
        castids = [a.find('a').attrs['href'][6:-1] for a in messy]
        moviedata.append(castids)
    except AttributeError:
        print('Unsuccessful: '+key)
        castbadmovies.append(key)
        continue
    castdata[key]=moviedata

        

print('completed but ' + str(len(castbadmovies)) + ' were unsuccesful!')    

Unsuccessful: tt8805968
Unsuccessful: m0001073
Unsuccessful: m0001073
Unsuccessful: m0000187
Unsuccessful: m0000187
Unsuccessful: m3119473
Unsuccessful: m3119473
Unsuccessful: tt7062794
Unsuccessful: m0000203
Unsuccessful: m0000203
Unsuccessful: tt3006462
Unsuccessful: tt1397132
Unsuccessful: m0004266
Unsuccessful: m0004266
Unsuccessful: m0502342
Unsuccessful: m0502342
Unsuccessful: tt8080292
Unsuccessful: m0290556
Unsuccessful: m0290556
Unsuccessful: tt3070498
Unsuccessful: m0000568
Unsuccessful: m0000568
Unsuccessful: m0396079
Unsuccessful: m0396079
Unsuccessful: m0908094
Unsuccessful: m0908094
Unsuccessful: tt6156584
Unsuccessful: tt6932244
Unsuccessful: tt0387199
Unsuccessful: tt5114356
Unsuccessful: tt0075488
Unsuccessful: m0005522
Unsuccessful: m0005522
Unsuccessful: m0000827
Unsuccessful: m0000827
Unsuccessful: tt1321865
Unsuccessful: tt5673782
Unsuccessful: tt2660806
Unsuccessful: m2442289
Unsuccessful: m2442289
Unsuccessful: tt1842530
Unsuccessful: m3507289
Unsuccessful: m3507

In [185]:
castdf = pd.DataFrame.from_dict(castdata, orient='index')
castdf.to_csv('castdata.csv')
castdf.head()


,0,1
tt1298650,"[Johnny Depp, Penélope Cruz, Geoffrey Rush, Ia...","[nm0000136, nm0004851, nm0001691, nm0574534, n..."
tt6565702,"[James McAvoy, Michael Fassbender, Jennifer La...","[nm0564215, nm1055413, nm2225369, nm0396558, n..."
tt2395427,"[Robert Downey Jr., Chris Hemsworth, Mark Ruff...","[nm0000375, nm1165110, nm0749263, nm0262635, n..."
tt2527336,"[Mark Hamill, Carrie Fisher, Adam Driver, Dais...","[nm0000434, nm0000402, nm3485845, nm5397459, n..."
tt2488496,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...","[nm0000148, nm0000434, nm0000402, nm3485845, n..."


In [207]:
castbaddf = pd.DataFrame(set(castbadmovies))
castbaddf.to_csv('castbad.csv')


In [255]:
len(castdf)

1687

In [195]:
roi.head()

,Unnamed: 0,level_0,index,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,dom_roi,world_roi,release_month,release_day,release_year,season,imdblink,imdbcode
0,0,0,1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,-0.412899,1.546673,May,20,2011,spring,/title/tt1298650/,tt1298650
1,1,1,2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.877822,-0.572108,Jun,7,2019,summer,/title/tt6565702/,tt6565702
2,2,2,3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,0.388403,3.243841,May,1,2015,spring,/title/tt2395427/,tt2395427
3,3,3,4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,0.956408,3.153696,Dec,15,2017,winter,/title/tt2527336/,tt2527336
4,4,4,5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2.060988,5.710167,Dec,18,2015,winter,/title/tt2488496/,tt2488496


In [250]:
castbaddf['title']=[" ".join(roi[roi.imdbcode==key].movie.to_string().split()[1:]) for key in castbaddf[0]]


In [129]:
#CAST
badmovies=[]
castdata= {}
moviedata=[] 
key = roi.imdbcode[random.randrange(0,len(roi.imdbcode))]
url = 'https://www.imdb.com/title/'+key+'/'
page = requests.get(url)
if page.status_code != requests.codes.ok:
    print('Unsuccessful connection')
soup=BeautifulSoup(page.content, 'html.parser')
print(url)
#findall 'td', class_='primaryphoto'
#findall 'img', attrs['title']
#edit for cast info here
messy = soup.find('table',class_='cast_list').find_all('td', class_=False)
cast= [actor.text.replace('\n','').strip() for actor in messy]
moviedata.append(cast)
castids = [a.find('a').attrs['href'][6:-1] for a in messy]
moviedata.append(castids)
castdata[key]= moviedata
print(castdata)

https://www.imdb.com/title/tt1931435/
{'tt1931435': [['Robert De Niro', 'Katherine Heigl', 'Diane Keaton', 'Amanda Seyfried', 'Topher Grace', 'Susan Sarandon', 'Robin Williams', 'Ben Barnes', 'Christine Ebersole', 'David Rasche', 'Patricia Rae', 'Ana Ayora', 'Kyle Bornheimer', 'Megan Ketch', 'Christa Campbell'], ['nm0000134', 'nm0001337', 'nm0000473', 'nm1086543', 'nm0333410', 'nm0000215', 'nm0000245', 'nm1602660', 'nm0002056', 'nm0711058', 'nm0990777', 'nm3065524', 'nm1531180', 'nm3430569', 'nm0132300']]}
